<a href="https://colab.research.google.com/github/zoeyxzong/ptsdfactors/blob/main/PTSDfactors_ZZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **A P-Technique Factor Analysis of Post-Traumatic Stress Disorder Symptoms**
### Zoe Zong | Digital Humanities 100 | June 2021


---



## **1. Library and Functions**



*   Load Packages



In [ ]:
install.packages('psych')
install.packages('glmnet')
install.packages('Hmisc')
install.packages('fmsb')
install.packages('DataCombine')
install.packages('tidyLPA')
install.packages('tidyverse')
install.packages('dplyr')
install.packages('mclust')
install.packages('mgm')
install.packages('qgraph')
install.packages('pROC')
install.packages('reshape2')
install.packages('graphicalVAR')

In [ ]:
rm(list=ls())
lagpad <- function(x, k) {
  c(rep(NA, k), x)[1 : length(x)] 
}  # used for creating lagged data structure

library(psych)
library(glmnet)
library(Hmisc)
library(fmsb)
library(DataCombine)
library(tidyLPA)
library(tidyverse)
library(dplyr)
library(mclust)
library(mgm)
library(qgraph)
library(pROC)
library(reshape2)
library(graphicalVAR)

ERROR: ignored

* beepday2consec

In [ ]:
beepday2consec <- function(beepvar, # Beep number in EMA study
                           dayvar) # Day Number
# a function that flags each timepoint as either ping 1/2/3/4, day#, and index of each ping out of the total # of pings
{
  
  # Input Checks
  if(!all(dayvar == round(dayvar))) stop("beepvar has to be a vector of non-negative integers")
  if(!all(beepvar == round(beepvar))) stop("beepvar has to be a vector of non-negative integers")
  if(length(beepvar) != length(dayvar)) stop("beepvar has to have the same length as dayvar")
  
  # Compute Aux variables
  fillin_beep <- max(beepvar) + 2 # fill in an integer for day/night shifts that ensures that lagData() treats this data point as non-consecutive
  n <- length(dayvar)
  
  # Get day breaks
  ind_sameday <- dayvar[-1] == dayvar[-n]
  ind_sameday[1] <- TRUE # dont change beep of first measurement
  
  consec <- rep(NA, n)
  consec[1] <- 1
  
  # check consecutiveness for each (consecutive) row-pair
  counter <- 1
  for(i in 2:n) {
    beep_diff <- beepvar[i] - beepvar[i-1]
    day_diff <- dayvar[i] - dayvar[i-1]
    if(beep_diff == 1 & day_diff == 0) counter <- counter + 1 else counter <- counter + 2
    consec[i] <- counter
  }
  
  return(consec)
  
}

* lagData

In [ ]:
#creates the lagged structure
lagData <- function(data, 
                    lags, 
                    consec = NULL) {
  
  
  # ---------- Compute Aux Variables ----------
  
  data <- as.matrix(data) # turn into matrix
  
  max_lag <- max(lags) # maximum lag
  lags_ext <- 1:max(lags) # makes it easier to delete right columns
  
  n <- nrow(data)
  p <- ncol(data)
  n_var <- nrow(data) - max(lags)
  n_lags <- length(lags_ext)
  
  data_response <- data #[-c(1:max_lag), ]
  
  if(!is.null(consec)) m_consec <- matrix(NA, 
                                          nrow = n, 
                                          ncol = n_lags)
  
  # ---------- Lag Variables ----------
  
  # Storage
  l_data_lags <- list()
  
  # Loop through lags
  lag_pos <- 1 # to make sure that the list is filled successively, if not a full sequence (e.g. lags = c(1,5)); otherwise this leads to problems later in the code  
  for(lag in lags) {
    
    lagged_data <- matrix(NA, nrow = n, ncol=p)
    lagged_data[(lag+1):n, ] <- data[-((n-lag+1) : n), ]
    lagged_data <- matrix(lagged_data, 
                          ncol = p, 
                          nrow = n) 
    colnames(lagged_data) <- paste("V", 1:p, '.lag', lag, '.', sep = "")
    
    l_data_lags[[lag_pos]] <- lagged_data
    
    lag_pos <- lag_pos + 1
  }
  
  # ---------- Knock Out if not consecutive ----------
  
  
  # browser()
  
  if(!is.null(consec)) {
    
    for(lag in lags_ext)  m_consec[(lag+1):n, lag] <- consec[-((n-lag+1) : n)]
    
    # Calculate which cases to knock out
    m_consec_check <- cbind(consec, m_consec)
    
    v_check <- apply(m_consec_check, 1, function(x) {
      
      if(any(is.na(x))) {
        FALSE
      } else {
        check_row <- x[1] - x[-1] == 1:length(x[-1]) # check for extended lags 1:max(lags)
        check_row_relevant <- check_row[lags_ext %in% lags] # but then compute check only over the lags that are actually specified
        if(any(check_row_relevant == FALSE)) FALSE else TRUE # and return test result: any required previous measurement missing? => FALSE
      }
      
    })
    
  } else {
    
    v_check <- rep(TRUE, n)
    v_check[1:n_lags] <- FALSE
    
  }
  
  
  # ---------- Output ----------
  
  outlist <- list()
  outlist$data_response <- data_response
  outlist$l_data_lags <- l_data_lags
  outlist$included <- v_check
  
  
  return(outlist)
  
}




---


## **2. Data Setup and Cleaning**



*   Read in and view data



In [ ]:
#Read in data
data <- read.csv("dat.csv", as.is=TRUE)

#let's check out the data
View(data)

Warning message in file(file, "rt"):
“cannot open file 'dat.csv': No such file or directory”


ERROR: ignored

* delete variables that were asked one time per day (these happen to be sleep variables):

In [ ]:

#TST= how many hours did you sleep last night
#SQ= experienced restless or unsatifysing sleep
#SOL= had trouble falling or staying asleep 

#keep sleep variables that were asked four times per day:
#dreams = had unpleasant dreams about a traumatic event
#sleepy = felt sleepy 

data[12:14] <- NULL

* Rename

In [ ]:
colnames(data) <- c("start","finish","enthus", "hyper", "agg", "fear", "self", "cogav", "relive", "dreams", "mem", "horror", "angry", "guilt", "shame", "other","world", "blameS", "blameO", "behav", "upset", "phys", "amnesia", "anhed", "distant", "difpos", "reck", "startle", "concen", "sleepy", "positive", "happy", "content", "calm", "fatigue","tense")

data$start <- as.character(data$start)
data$finish <- as.character(data$finish)

View(data)

* More Data Cleaning

In [ ]:


#Duplicate and lag time
data$lag=lagpad(data$start,1)

#Calculate time differences
data$tdif=as.numeric(difftime(strptime(data$start,"%m/%d/%Y %H:%M"),strptime(data$lag,"%m/%d/%Y %H:%M")))

#Replace NA
data$tdif[is.na(data$tdif)]<- 0

#Calculate cumulative sum of numeric elapsed time
data$cumsumT=cumsum(data$tdif)

# how many obs, and how many are complete
nrow(data) #124
length(which(complete.cases(data[,3]))) #104

#Trim time series to even 4obs/day
#first visualize and determine where to trim.
options(width=90)
data$start
dati=data
dati$start
length(which(complete.cases(dati[,3])))

#Use for internal missing rows: add a row where it's missing
new <- rep(NA, length(dati))
dati <- InsertRow(dati, NewRow=new, RowNum = 1)
dati <- InsertRow(dati, NewRow=new)
dati <- InsertRow(dati, NewRow=new)
dati <- InsertRow(dati, NewRow=new)
dati$start

#Code days of the week
dati$day <- rep(1:7, each=4, length.out=nrow(dati)) #https://lubridate.tidyverse.org/ - need to convert 'time' column into lubridate formate (can also use this to calculate time differences)
# 1 = Tuesday
dati$mon=ifelse(dati$day==7,1,0)
dati$tues=ifelse(dati$day==1,1,0)
dati$wed=ifelse(dati$day==2,1,0)
dati$thur=ifelse(dati$day==3,1,0)
dati$fri=ifelse(dati$day==4,1,0)
dati$sat=ifelse(dati$day==5,1,0)
dati$sun=ifelse(dati$day==6,1,0)

#Code pings
dati$ping=seq(0,3,1)
dati$morning=ifelse(dati$ping==0,1,0)
dati$midday=ifelse(dati$ping==1,1,0)
dati$eve=ifelse(dati$ping==2,1,0)
dati$night=ifelse(dati$ping==3,1,0)
datx=dati

#Temporal variables
datx$linear=scale(datx$cumsumT)
datx$quad=datx$linear^2
datx$cub=datx$linear^3
datx$cosT=cos(((2*pi)/24)*datx$cumsumT)
datx$sinT=sin(((2*pi)/24)*datx$cumsumT)
datx$cos2T=cos(((2*pi)/12)*datx$cumsumT)
datx$sin2T=sin(((2*pi)/12)*datx$cumsumT)
datx$cosW=cos(((2*pi)/168)*datx$cumsumT)
datx$sinW=sin(((2*pi)/168)*datx$cumsumT)

#Index consecutive measurements by ping and day
datx$dayvar=rep(1:(nrow(datx)/4),each=4)
datx$beepvar=datx$ping+1

#Create filter to mark cases with missing data
datx$filter=ifelse(!complete.cases(datx[,c(3:24)]),1,0)

## Remove NAs, suppress row names
daty=subset(datx,filter==0)
row.names(daty)<- NULL



* Check out the cleaned data

In [ ]:
View(daty)

## **3. Exploratory Factor Analysis**

## **4. Confirmatory Factor Analysis**

## **5. Visualization**

## **6. Summary**